In [1]:
# Only import neccessary packages
import pandas as pd
import sys
sys.path.append('../../')
import csky as cy
from greco_grb.scripts import SETTING
paths = SETTING.PATH()
print(paths)
USER = paths.USER
ICDATA_DIR = paths.ICDATA_DIR
DATA_DIR = paths.DATA_DIR
ANA_DIR = paths.ANA_DIR
from greco_grb.scripts.utils import *
from glob import glob
import histlite as hl
import scipy


#### Working at GT ####

LOCATION: 	 GT
USER: 	 cchen641
ICDATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/candidate-002-p10
DATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data
ANA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output



In [2]:
UNBLIND_PATH = "/storage/home/hhive1/cchen641/icecube/greco_grb/data/csky_output/unblind"
UNBLIND_RESULT = f"{UNBLIND_PATH}/result02.pkl"

In [3]:
result02 = pd.read_pickle(UNBLIND_RESULT)


In [4]:
result02.columns

Index(['grb_name', 'grb_name_GBM', 'ra', 'dec', 'source_uncertainty',
       'mjd_start', 't100', 't90', 'gbm', 'ra_source', 'decl_source',
       'pos_error_source', 'fluence', 'redshift', 't_center', 'hasHealpix',
       'tw', 'ts', 'ns', 'pre_p', 'post_p', 'mjd_start_utc'],
      dtype='object')

# Unblind SoB at each TW
For each GRB, specify TW = 10, 25, 50, 100, 250, 500 and **prompt (T100)**, get the unblind events SoB

In [5]:
print("\n===== Setting up csky =====\n")
data_dir = ICDATA_DIR
data_filenames = sorted(glob(data_dir + '/IC86_20*.data.npy'))
sig_filenames = sorted(glob(data_dir + '/IC86_2012.nu*_merged.npy'))
grl_filenames = sorted(glob(data_dir + '/GRL/IC86_20*.data.npy'))

################ energy lower bound #############
min_log_e = np.log10(10)
#################################################
bins_sindec = np.linspace(-1, 1, 25+1)  
bins_logenergy = np.linspace(min_log_e, 4, 25+1)

data = [np.load(data_filename) for data_filename in data_filenames]
data = np.hstack(data)
sig = [np.load(sig_filename) for sig_filename in sig_filenames]
sig = np.hstack(sig)
grl = [np.load(grl_filename) for grl_filename in grl_filenames]
grl = np.hstack(grl)
if min_log_e is not None:
    data_mask = data['logE'] > min_log_e
    data = data[data_mask]
    sig_mask = sig['logE'] > min_log_e
    sig = sig[sig_mask]
    
dataset_spec = cy.selections.CustomDataSpecs.CustomDataSpec(data, sig, np.sum(grl['livetime']),
                                                     sindec_bins=bins_sindec,
                                                     logenergy_bins=bins_logenergy,
                                                     grl=grl, key='greco_v2.10', cascades=True)

ANA_DIR = cy.utils.ensure_dir(ANA_DIR)
# on OSG
# ana_dir = "./"
ana = cy.get_analysis(cy.selections.repo
                      , dataset_spec
                      , dir=ANA_DIR
                      , load_sig=True)  # to save memory: use False



===== Setting up csky =====

Setting up Analysis for:
greco_v2.10
Setting up greco_v2.10...
Energy PDF Ratio Model...
  * gamma = 4.0000 ...
Signal Acceptance Model...
  * gamma = 4.0000 ...
Done.


In [131]:
class args:
    grb_idx_start = 0 
    grb_idx_end = 1 
    ncpu = 24         ################### Number of Cores #############
    out_dir = "/storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output"
############# used for spatial_prior_trial_runner
conf = {
    'ana': ana,
    #### llh basics: csky.conf
    'space': 'ps', # ps/fitps/template/prior
    'time': 'transient', # utf/lc/transient
    'energy': 'customflux', # fit/customflux
    'flux': cy.hyp.PowerLawFlux(2.5),
    #### inj.py - prior has some duplications against space's prior
    'sig': 'transient', # ps/tw/lc/transient/template/prior
    'full_sky': True,
    'extended': True,
    'mp_cpus': args.ncpu,
    'cut_n_sigma': 3,
}

cy.CONF.update(conf)

In [7]:
df = pd.read_pickle(DATA_DIR+"/grbwebgbm/grbweb_gbm_noHealpix_2268.pkl")

In [8]:
df

,grb_name,grb_name_GBM,ra,dec,source_uncertainty,mjd_start,t100,t90,gbm,ra_source,decl_source,pos_error_source,fluence,redshift,t_center,hasHealpix
0,GRB200529B,GRB200529504,337.200000,55.230000,3.506136,58998.503432,0.000539,0.000539,1,Fermi_GBM,Fermi_GBM,Fermi_GBM,5.001000e-06,NaN,58998.503701,True
1,GRB200529A,GRB200529039,238.763917,-11.073639,0.000233,58998.039294,0.001004,0.000984,0,Swift,Swift,Swift,4.847700e-06,NaN,58998.039796,True
2,GRB200528A,GRB200528436,176.637458,58.192583,0.000194,58997.435683,0.000700,0.000655,0,Swift,Swift,Swift,2.905200e-05,NaN,58997.436033,True
3,GRB200525A,GRB200525611,20.610000,-8.520000,4.536708,58994.611328,0.001081,0.001081,1,Fermi_GBM,Fermi_GBM,Fermi_GBM,8.858900e-06,NaN,58994.611869,True
4,GRB200524A,GRB200524211,212.750000,60.966667,0.100000,58993.211115,0.000445,0.000437,0,IPN,IPN,IPN,1.396400e-05,1.256,58993.211338,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2263,GRB120429B,GRB120429484,133.040000,-32.230000,6.505598,56046.484059,0.000178,0.000178,1,Fermi_GBM,Fermi_GBM,Fermi_GBM,2.367800e-06,NaN,56046.484148,True
2264,GRB120429A,GRB120429003,165.980000,-8.760000,9.378528,56046.002860,0.000019,0.000019,1,Fermi_GBM,Fermi_GBM,Fermi_GBM,2.793700e-07,NaN,56046.002869,True
2265,GRB120427B,GRB120427153,114.700000,50.210000,15.003090,56044.153189,0.000264,0.000264,1,Fermi_GBM,Fermi_GBM,Fermi_GBM,6.805100e-07,NaN,56044.153321,True
2266,GRB120427A,GRB120427054,224.935000,29.311000,NaN,56044.053773,0.000089,0.000062,0,"GCN_circulars,IPN Triangulation","GCN_circulars,IPN Triangulation",None,7.429900e-06,NaN,56044.053818,True


## 1. get s/b and n_b

In [9]:
## only uncomment below if want refreshed results

In [141]:
# sob_matrix = [
#     [],  # t100
#     [],  # 10 
#     [],  # 25
#     [],  # 50
#     [],  # 100
#     [],  # 250
#     [],  # 500
# ]
# n_b_list = [0,0,0,0,0,0,0]  # expect 1+6 values
# # for each grb
# # for grb_idx in range(args.grb_idx_start, args.grb_idx_end):
# timer = cy.timing.Timer()
# time = timer.time
# with time("Test..."):
#     end_idx_plus_1 = 2268 # 2268
#     for grb_idx in range(0, end_idx_plus_1):
#         # load grb info
#         grb_name = df.grb_name[grb_idx]
#         if(grb_idx % 20 == 0):
#             print(f"(report every 20) Working on {grb_name}")
#         grb_row = df.loc[df['grb_name'] == grb_name]
#         ra = grb_row.ra
#         dec = grb_row.dec
#         t100 = grb_row.t100 * 86400.
#         # load grb healpix map
#         healpix = np.load(DATA_DIR+"/grbwebgbm/healpix/{}_healpix_nside64.npy".format(grb_name))
#         healpix = np.maximum(healpix,0)
#         ########## healpix reduce (< instead of <=) ##########
#         healpix[healpix < isf_healpix(healpix, q=0.99)] = 0
#         healpix = healpix / np.sum(healpix)
#         # for each tw_in_second
#     #     tss, nss = [], []
#         for tw_idx, tw_in_second in enumerate([t100, 10,25,50,100,250,500]):
#             tw = tw_in_second/86400.
#             tw_start = grb_row.t_center - 0.5*tw
#             src = cy.sources(
#                 ra=ra,
#                 dec=dec,
#                 deg=True,
#                 mjd=tw_start, 
#                 sigma_t=np.zeros_like(tw), 
#                 t_100=tw,  # in days
#                 prior=[hl.heal.HealHist(healpix)],
#                 name=grb_name
#             )
#             tr = cy.get_trial_runner(conf=cy.CONF, ana=ana, src=src)
#             L = tr.get_one_llh(TRUTH=True)
#             #### i=0 for bg, i=1 for sig; unblind so i=0.
#             SB_space = cy.inspect.get_space_eval(L, -1, 0)()[0]
#             ## unblinded, no need for (gamma=xxx)
#             SB_energy = cy.inspect.get_energy_eval(L, -1, 0)()[0]
#             SB = SB_space * SB_energy
            
#             sob_matrix[tw_idx].append(SB)
#             n_b_list[tw_idx] += L.llh_model.N_bg
            
# with time("Saving..."):
#     np.save(DATA_DIR+f"/csky_output/comparison/unblind/{end_idx_plus_1}_sob_matrix.npy", 
#                 np.array(sob_matrix,dtype=object))
#     np.save(DATA_DIR+f"/csky_output/comparison/unblind/{end_idx_plus_1}_n_b_list.npy", 
#                 np.array(n_b_list,dtype=object))

## 2. get `n_fit` and `TS_unblind` from `s/b` and `n_b`

In [142]:
end_idx_plus_1 = 2268
sob_matrix_from_file = np.load(DATA_DIR+f"/csky_output/comparison/unblind/{end_idx_plus_1}_sob_matrix.npy", allow_pickle=True)
n_b_list_from_file = np.load(DATA_DIR+f"/csky_output/comparison/unblind/{end_idx_plus_1}_n_b_list.npy", allow_pickle=True)


In [144]:
sob_matrix_from_file, n_b_list_from_file

(array([[array([], dtype=float64), array([], dtype=float64),
         array([], dtype=float64), ..., array([], dtype=float64),
         array([], dtype=float64), array([], dtype=float64)],
        [array([], dtype=float64), array([], dtype=float64),
         array([], dtype=float64), ..., array([], dtype=float64),
         array([], dtype=float64), array([], dtype=float64)],
        [array([], dtype=float64), array([], dtype=float64),
         array([], dtype=float64), ..., array([], dtype=float64),
         array([], dtype=float64), array([], dtype=float64)],
        ...,
        [array([], dtype=float64), array([], dtype=float64),
         array([], dtype=float64), ..., array([], dtype=float64),
         array([0.42919967]), array([], dtype=float64)],
        [array([0.40161226]), array([0.37091752]),
         array([], dtype=float64), ..., array([], dtype=float64),
         array([0.42919967]), array([], dtype=float64)],
        [array([0.40161226]), array([1.7707964 , 0.37091752]),

In [13]:
###########################
#                         #
#        Functions        #
#                         #
###########################


def neg_TS(n_s, sobe_list, n_b):
    """
    Calculate the test statistic for a given S/B list, 
        number of background events, and n_s.
        
    Define the function for use in a minimizer later.
    
    Returns the negative TS
    """
    TS = -n_s + np.sum(np.log(n_s * (sobe_list/n_b) +1))
    
    return -TS

def calc_stacked_TS(sobe_list, n_b):
    """
    Calculate the TS for a set of S/B and a stacked n_b.  
        This function calls a minimizer to find the n_s 
        that minimizes the negative TS value.
        
    sobe_list: numpy array
        Concatenated list of S/B values for all GRBs.  These 
        are the signal/background values for every event 
        contributing to a GRB in a trial
    n_b: float
        Number of expected background events for the entire 
        stacked time window
    
    Returns a test statistic and fit ns value for the trial.
    """
    
    # See the last cell of this script for several examples of how to make this work

    # A short function to sort out passed args versus what the minimizer should fit
    def fit_neg_ts(n_s): return neg_TS(n_s, sobe_list, n_b)
    
    # Fit the results with a scipy minimizer
    fit_results = scipy.optimize.minimize(fit_neg_ts, [1.], bounds=[(0.,len(sobe_list))]) 

    # Get the fit results
    ns_fit = fit_results.x[0]
    TS = -fit_results.fun[0]
    
    return TS, ns_fit



def calc_unblind_TS(sobe_list, n_b):
    """
    Calculate the unblinded TS for some subset of GRBs
    
    grb_results_array: numpy array
        Sky and duration cuts to determine which GRBs are considered
    TW_str: str
        Prompt or 0-9 for time window
        
    Parameter:
    ---------------
    sobe_list : numpy array
                the length of the list can be from 0 to a number greater than 2268
                
    n_b : float
            one number obtained from all 2268 GRBs
    """
    
    if len(sobe_list) != 0:
        sobe_list_combined = np.concatenate(sobe_list)
    else:
        sobe_list_combined = np.array([0.])

    #n_b = total_time*0.007  # Fixed background rate so calculate n_b easily
    
    TS_unblind, ns_fit = calc_stacked_TS(sobe_list_combined, n_b)
    if TS_unblind < 0:
        TS_unblind = 0.
        
    return TS_unblind, ns_fit

In [14]:
sob_matrix_from_file.shape, n_b_list_from_file.shape

((7, 2268), (7,))

In [15]:
# [t100, 10,25,50,100,250,500]
[calc_unblind_TS(sob_matrix_from_file[i], n_b_list_from_file[i]) for i in range(0, 7)]

[(0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.06077506920024356, 1.5388555114402187),
 (0.0, 0.0),
 (0.0, 0.0)]

# 3. Get sob, n_b and stacked TS from background runs

In [133]:
class args:
    grb_name = "GRB180423A"    # real healpix example
    # grb_name = "GRB190415A"    # fake healpix example 
    batchNtrials = 5
    batchIndex = 0
    ncpu = 24
    outfilename = ""

In [134]:
rng=np.random.default_rng(abs(java_hash(args.grb_name)))
seeds = rng.integers(int(1e9), size=int(2e8))[args.batchNtrials*args.batchIndex: args.batchNtrials*(args.batchIndex + 1)]
print("\n...Done\n")


...Done



In [135]:
seeds

array([840444166, 348822262, 164807178, 105228374,  13675713])

In [169]:
grb_row.t100.values[0]

0.000634074074074074

In [170]:
grb_name = args.grb_name
grb_row = df.loc[df['grb_name'] == grb_name]
ra = grb_row.ra
dec = grb_row.dec
t100 = grb_row.t100.values[0] * 86400.
### load grb healpix map
healpix = np.load(DATA_DIR+"/grbwebgbm/healpix/{}_healpix_nside64.npy".format(grb_name))
healpix = np.maximum(healpix,0)
########## healpix reduce (< instead of <=) ##########
healpix[healpix < isf_healpix(healpix, q=0.99)] = 0
healpix = healpix / np.sum(healpix)
### for each tw_in_second
for tw_idx, tw_in_second in enumerate([t100, 10,25,50,100,250,500]):
    timer = cy.timing.Timer()
    time = timer.time
    with time(f"Working on {grb_name} with TW{tw_in_second}..."):
        sob_list = []
        n_b_list =[]
        tw = tw_in_second/86400.
        tw_start = grb_row.t_center - 0.5*tw
        src = cy.sources(
            ra=ra,
            dec=dec,
            deg=True,
            mjd=tw_start, 
            sigma_t=np.zeros_like(tw), 
            t_100=tw,  # in days
            prior=[hl.heal.HealHist(healpix)],
            name=grb_name
        )
        for no_trial in range(args.batchNtrials):
            sptr = cy.get_spatial_prior_trial_runner(conf=cy.CONF
                                         ,src_tr=src
                                         ,llh_priors=[healpix])

            ############## ref: osg_stacked_bg_tsd.py ########################
            ############## Get the pixelmax ##################################
            try:
                _, ts, ns, ra_fit, dec_fit = sptr.get_one_fit(TRUTH=False, 
                                                               mp_cpus=args.ncpu, 
                                                               logging=False,
                                                              seed=seeds[no_trial])
            except:
                ts, ns, ra_fit, dec_fit = 0.0, 0.0, 0.0, 0.0
            if ns == 0.0:
                ra_fit, dec_fit = grb_row.ra, grb_row.dec
            ############## Get SoB, n_b for point source: pixelmax ##########
            src_fit = cy.sources(
                            ra=ra_fit,
                            dec=dec_fit,
                            deg=True,
                            mjd=tw_start, 
                            sigma_t=np.zeros_like(tw), 
                            t_100=tw,  # in days
                            # prior=[hl.heal.HealHist(healpix)], # not important here
                            name=grb_name
                        )
            tr = cy.get_trial_runner(conf=cy.CONF, ana=ana, src=src_fit)
            L = tr.get_one_llh(TRUTH=False, seed=seeds[no_trial])
            try:
                #### i=0 for bg, i=1 for sig; unblind so i=0.
                SB_space = cy.inspect.get_space_eval(L, -1, 0)()[0]
                ## unblinded, no need for (gamma=xxx)
                SB_energy = cy.inspect.get_energy_eval(L, -1, 0)()[0]
                SB = SB_space * SB_energy
            except:
                SB = np.array([])
            n_b = L.llh_model.N_bg
            sob_list.append(SB)
            n_b_list.append(n_b)
    # Save sob_list and n_b_list for this GRB with this TW
    print(f"Saving SoB and n_b for {grb_name} with TW={tw_in_second:.2f}")
    output_path = cy.utils.ensure_dir(DATA_DIR+f"/csky_output/comparison/background/{grb_name}")
    np.save(f"{output_path}/{grb_name}_tw{tw_in_second:.2f}_batchIndex{args.batchIndex}_batchNtrials{args.batchNtrials}.npy", np.array(sob_list))
    np.save(f"{output_path}/{grb_name}_tw{tw_in_second:.2f}_batchIndex{args.batchIndex}_batchNtrials{args.batchNtrials}.npy", np.array(n_b_list))


0:00:18.495897 elapsed.
Saving SoB and n_b for GRB180423A with TW=54.78

0:00:18.361897 elapsed.
Saving SoB and n_b for GRB180423A with TW=10.00

0:00:18.257248 elapsed.
Saving SoB and n_b for GRB180423A with TW=25.00

0:00:18.374427 elapsed.
Saving SoB and n_b for GRB180423A with TW=50.00

0:00:27.236933 elapsed.
Saving SoB and n_b for GRB180423A with TW=100.00


/storage/home/hhive1/cchen641/.conda/envs/icecube_py3/lib/python3.7/site-packages/ipykernel_launcher.py:74: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



0:00:33.152332 elapsed.
Saving SoB and n_b for GRB180423A with TW=250.00

0:00:42.305993 elapsed.
Saving SoB and n_b for GRB180423A with TW=500.00


In [140]:
n_b_list

[2080.6113763385733,
 2080.6113763385733,
 2080.6113763385733,
 2080.6113763385733,
 2080.6113763385733]

/storage/home/hhive1/cchen641/.conda/envs/icecube_py3/lib/python3.7/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


# 4. Calculate TS_upper_limit based on TS_median from background and TS_unblind above